# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown
from omegaconf import OmegaConf
import datetime,json, os, requests, re
from openai import OpenAI
from tqdm import tqdm

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-08-06'

## API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api
- `ASKNEWS_CLIENT_ID`, `ASKNEWS_SECRET`

In [3]:
token_fn = "tokens.yaml"

In [4]:
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

## LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [5]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

## IFP

In [6]:
class IFP:

    forecast_fields = ['question_id',
                       'title',
                       'feedback',
                       'resolution_criteria', 
                       'background', 
                       'event', 
                       'model_domain']

    forecast_format = f"|{'|'.join(forecast_fields)}|"

    openai_max_tokens = 30000

    def format(self):
        rec = vars(self)
        fmt = '|'.join([f"{x}: {rec[x]}" for x in self.forecast_fields])
        return '|' + fmt + '|'

    def over_openai_max(self):
        sep = self.format()
        return len(sep.split(' ')) > self.openai_max_tokens # Max limit for OpenAI

    def record(self):
        return self.format()
        if self.over_openai_max(): 
            self.news = ' '.join(self.news.split(' ')[0:int(0.5*self.openai_max_tokens)])
        if self.over_openai_max(): 
            self.research = ' '.join(self.research.split(' ')[0:int(0.5*self.openai_max_tokens)])
        if self.over_openai_max(): 
            raise Exception('IFP record over OpenAI max')
        return self.format()

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]
        self.event = ''
        self.model_domain = ''
        self.feedback = ''

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

    def upload(self):
        post_question_prediction(self.question_id, self.forecast)
        post_question_comment(self.question_id, self.rationale)

In [7]:
unit_test = False

In [8]:
if unit_test:
    qid = 26775
    ifp = IFP(qid)
    ifps = {qid: ifp}
    print(ifp.record())

## News source (AskNews)

https://colab.research.google.com/drive/1tc383HraMZOiyfKFF1EXAtlTYbsuv3Q5?usp=sharing

In [9]:
from asknews_sdk import AskNewsSDK

def asknews_to_dict(x):
    return dict([x.split(': ', 1) for x in x.split('\n')])

class AskNews():

    def __init__(self):
        ASKNEWS_CLIENT_ID = config.ASKNEWS_CLIENT_ID
        ASKNEWS_SECRET = config.ASKNEWS_SECRET
        
        self.ask = AskNewsSDK(
              client_id=config.ASKNEWS_CLIENT_ID,
              client_secret=config.ASKNEWS_SECRET,
              scopes=["news"]
          )

        self.cache = {}
        
    def query(self, 
              q, 
              method,    # use "nl" for natural language for your search, or "kw" for keyword, or 'both'
              strategy): # strategy="latest news" enforces looking at the latest news only
                         # strategy="news knowledge" looks for relevant news within the past 60 days
        return self.ask.news.search_news(
            query=q, # your keyword query
            n_articles=10, # control the number of articles to include in the context
            return_type="dicts",  # you can also ask for "dicts" if you want more information
            method=method, 
            strategy=strategy).as_dicts # strategy="latest news" enforces looking at the latest news only,

    def multi_strategy(self, q):
        all = []
        for strategy in ['latest news', 'news knowledge']:
            for x in self.query(q, 'both', strategy):
                all.append(x.summary)
        all = list(set(all))
        return '\n\n'.join(all)

    def research(self, event, group):
        q = '\n'.join(ifp.title.strip() for ifp in group)
        try:
            return self.cache[q]
        except:
            news = self.multi_strategy(q)
            self.cache[q] = news
        return news

In [10]:
if unit_test:
    ask = AskNews()
    event = '2024 Grand Chess Tour'
    group = [IFP(id) for id in [26771, 26772, 26773, 26774]]
    news = ask.research(event, group)
    print(news)

## LLMs

### In general

In [11]:
class LLM:
    def __init__(self, system_role):
        self.system_role = system_role
        self.messages = [{"role": "system", "content": system_role}]
        self.cache = {}

    def chat(self, query):
        try:
            return self.cache[query]
        except:
            self.messages.append({"role": "user", "content": query})
            text = self.message()
            self.messages.append({"role": "assistant", "content": text})
            self.cache[query] = text
            return text

### MetaAI

https://pypi.org/project/meta-ai-api/1.0.6/

In [12]:
from meta_ai_api import MetaAI as mai

In [13]:
class MetaAI(LLM):

    def __init__(self, system_role = "Nice talker"):
        self.ai = mai()
        super().__init__(system_role)
        
    def message(self):
        self.response = self.ai.prompt(message=self.messages[-1]['content'])
        return self.response['message']

In [14]:
if unit_test:
    ai = MetaAI()
    r = ai.chat(ifp.title)
    print(r)

### HuggingChat

https://pypi.org/project/hugchat/

In [15]:
from hugchat import hugchat
from hugchat.login import Login

In [16]:
class HuggingChat(LLM):

    def __init__(self, system_role = "Nice talker"):
        # Log in to huggingface and grant authorization to huggingchat
        EMAIL = config.HUGGINGFACE_USERNAME
        PASSWD = config.HUGGINGFACE_PASSWORD
        cookie_path_dir = "./cookies/" # NOTE: trailing slash (/) is required to avoid errors
        sign = Login(EMAIL, PASSWD)
        cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)
        self.chatbot = hugchat.ChatBot(cookies=cookies.get_dict()) 
        super().__init__(system_role)
        
    def message(self):
        result = ''.join([x['token'] for x in self.chatbot.chat(self.system_role + '\n\n' + self.messages[-1]['content']) if x])
        return result

    def web_search(self, query):
        query_result = self.chatbot.query(query, web_search=True)
        return query_result
        print(query_result)
        for source in query_result.web_search_sources:
            print(source.link)
            print(source.title)
            print(source.hostname)

    def available_models(self):
        return [(i,str(x)) for i,x in enumerate(self.chatbot.get_available_llm_models())]

    def switch_llm(self, i):
        self.chatbot.switch_llm(i)

    def conversation_info(self):
        info = self.chatbot.get_conversation_info()
        # print(info.id, info.title, info.model, info.system_prompt, info.history)
        return info

In [17]:
if unit_test:
    hc = HuggingChat()
    hc.switch_llm(1)
    print(hc.chat(ifp.title))

### Metaculus Anthropic

In [18]:
import json

class Claude(LLM):
       
    def message(self):
        content = self.messages[-1]['content']
        url = "https://www.metaculus.com/proxy/anthropic/v1/messages"
        headers =  {"Authorization": f"Token {config.METACULUS_TOKEN}",
                    "anthropic-version": "2023-06-01",
                    "content-type": "application/json"}
        data = {"model": "claude-3-5-sonnet-20240620",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": self.system_role + '\n\n' + content}]}
        response = requests.post(url, headers=headers, data = json.dumps(data))
        # Check the response status code
        if response.status_code != 200:
            print("Request failed with status code", response.status_code)
            print(response.text)
        return response.json()['content'][0]['text']

In [19]:
if unit_test:
    claude = Claude('Please help me.')
    print(claude.chat(ifp.title))

### Perplexity

In [20]:
class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

In [21]:
if unit_test:
    perp = Perplexity('You are the most deeply knowledgeable web search engine and insightful analyst')
    print(perp.chat(ifp.title))

### Metaculus OpenAI

In [22]:
import json

class MetacGPT(LLM):

    def __init__(self, system_role):
        super().__init__(system_role)
        
    def message(self):
        url = "https://www.metaculus.com/proxy/openai/v1/chat/completions"
        headers =  {"Authorization": f"Token {config.METACULUS_TOKEN}",
                    "content-type": "application/json"}
        data = {"model": "gpt-4o",
                "max_tokens": 1024,
                "messages": self.messages}
        response = requests.post(url, headers=headers, data = json.dumps(data))
        # Check the response status code
        if response.status_code != 200:
            print("Request failed with status code", response.status_code)
            print(response.text)
        self.rec = response.json()
        return self.rec ['choices'][0]['message']['content']

In [23]:
if unit_test:
    mgpt = MetacGPT('Please help me')
    print(mgpt.chat('Bruce Bueno de Mesquita'))

### OpenAI

In [24]:
class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

In [25]:
if unit_test:
    oai = ChatGPT('You are the most deeply knowledgeable web search engine and insightful analyst')
    print(oai.chat(ifp.title))

## Agents

#### Agent

In [26]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

#### Question correlator for common topic inference

In [27]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        print(KL)
        K1 = [x.split('|') for x in KL.split('\n')]
        self.topic_map = [(int(id),event) for _,event,id,_,_ in K1]
        self.topics = set([event for id,event in self.topic_map])
        self.topic_groups = {topic: [ifps[x] for x,y in self.topic_map if y == topic] for topic in self.topics}
        for id,event in self.topic_map :
            ifps[id].event = event
        return self.topic_groups

In [28]:
if unit_test:
    try:
        ifps
    except:
        test_day = '01AUG24'
        ifps = {id: IFP(id) for id in history[test_day]}
    qr = QuestionRelator(ChatGPT)
    print(qr.relate(ifps))

#### Model domain classifier

In [29]:
class ModelDomainClassifier(Agent):
    def __init__(self, llm):
        self.system_role = f"""
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will report the model domain of the question, one of

Rate over time
Election outcome
Sports performance
Military conflict
Civil unrest
Dated product announcement
Market price
Macroeconomics
Epidemic
Disease
Drug discovery
Medical device
Crime
Leadership change
Other
"""
        super().__init__(self.system_role, llm)

    def classify(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.model_domain = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

In [30]:
if unit_test:
    mdc = ModelDomainClassifier(ChatGPT)
    mdc.classify(ifp)

#### Rate Analyzer

In [31]:
class RateAnalyzer(Agent):
    def __init__(self, llm):
        self.system_role = f"""
A group of events are given, formatted as "GROUP" followed by a group of questions separated by carriage returns, followed by "NEWS" followed by news on the topic.
Today's date is {today}.
If the question is about a level that changes at some rate over time, you will report 
1. today's date, 
2. the end date of the question, 
3. the time in days D from today to end date, 
4. the daily rate of change R of the quantity,
5. today's value V of the quantity,
6. the change in value dV of the quantity = D * r,
7. the final value of the quantity F = V + dV.  Give your best estimate.
Otherwise if the question is about the date that an event will occur, you will report
1. today's date, 
2. the end (measurement) date of the question, 
3. the time in days D from today to end date, 
4. the specific date at which the event is most likely to occur, without reference to the end date of the question
5. whether the likely event date is before, or on or after the question end date

Provide only one type of answer, not both.
"""
        super().__init__(self.system_role, llm)

    def assess(self, group, news):
        titles = '\n'.join([ifp.title.strip() for ifp in group])
        self.q = f'GROUP\n{titles}\nNEWS\n{news}'
        self.rates = self.chat(self.q)
        return self.rates

In [32]:
if unit_test:
    try:
        group
    except:
        group = [IFP(id) for id in [26771, 26772, 26773, 26774]]

    rates = RateAnalyzer(ChatGPT)
    r = rates.assess(group, news)
    print(r)

#### Superforecaster

In [33]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as GROUP|group_description followed by NEWS|group_news followed by RATES|group_rates 
followed by the questions in the group formatted QUESTIONS followed by, for each question, {IFP.forecast_format}.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, event, news, rates, group):
        ifps = '\n'.join([ifp.record() for ifp in group])
        prompt = f"GROUP|{event}\nNEWS|{news}\nRATES|{rates}\nQUESTIONS\n{ifps}"
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        self.ifps = {ifp.question_id: ifp for ifp in group}
        for id, forecast, rationale in self.F4:
            self.ifps[id].forecast = forecast
            self.ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

In [34]:
if unit_test:
    sf = Superforecaster(ChatGPT)
    event = '2024 Grand Chess Tour'
    group = qr.topic_groups[event]
    sf.forecast(event, news, r, group)

#### Critic

In [35]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        self.fb = 'I prasldfkbeddy.'
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|"
        self.fb = self.chat(prompt)
        if '|' in self.fb: # LLM obeyed system role
            self.fb1 = self.fb.split('|')[2:]
            self.fb = ''.join([x.strip() for x in self.fb1 if x])
        ifp.feedback = self.fb 
        print(ifp.question_id, ifp.feedback)

In [36]:
if unit_test:
    for llm in [MetaAI, Claude, Perplexity, ChatGPT, HuggingChat]:
        print(llm.__name__)
        critic = Critic(llm)
        critic.feedback(ifp) 
        print()

#### Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

## IFP history

In [37]:
history = { '08JUL24': [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872],
            '09JUL24': [26006, 25936, 25935, 25934, 25933, 26004, 26005],
            '10JUL24': [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958],
            '11JUL24': [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024],
            '12JUL24': [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102],
            '15JUL24': [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159],
            '16JUL24': [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196],
            '17JUL24': [26210, 26211, 26212, 26213, 26214, 26215, 26216],
            '18JUL24': [26232, 26233, 26234, 26235, 26236],
            '19JUL24': [26302, 26303, 26304, 26305, 26306, 26307],
            '22JUL24': [26387, 26388, 26389, 26390, 26391, 26392],
            '23JUL24': [26404, 26405, 26406, 26407, 26408],
            '24JUL24': [26550, 26551, 26552, 26553, 26554, 26555],
            '25JUL24': [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577],
            '26JUL24': [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646],
            '29JUL24': [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683],
            '30JUL24': [26700, 26701, 26702, 26703, 26704, 26705, 26706],
            '31JUL24': [26816, 26817, 26818, 26819, 26820, 26821, 26844],
            '01AUG24': [26771, 26772, 26773, 26774, 26775, 26776, 26777, 26778, 26779, 26780, 26781],
            '02AUG24': [26837, 26838, 26839, 26840, 26841, 26842],
            '05AUG24': [26913, 26914, 26915, 26916, 26917, 26918, 26919, 26920],
            '06AUG24': [26957, 27019, 27020, 27021, 27022]}

In [38]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
print(today, today_ids)

2024-08-06 [26957, 27019, 27020, 27021, 27022]


In [39]:
ifps = {id: IFP(id) for id in today_ids}

## Forecasting process

In [40]:
class Forecaster:
    
    def __init__(self, ifps):
        self.max_tries = 4
        self.qr = QuestionRelator(ChatGPT)

        self.ask = AskNews()
        self.critic = Critic(HuggingChat)
        self.mdc = ModelDomainClassifier(ChatGPT)
        self.ifps = ifps
        
    def fit(self):
        print('MODEL DOMAIN')
        for ifp in self.ifps.values():
            self.mdc.classify(ifp)
        print()  
        print('CORRELATOR')
        self.rho = self.qr.relate(self.ifps)
        for event, group in self.rho.items():
            news = self.ask.research(event, group)    
            ra = RateAnalyzer(ChatGPT)
            rates = ra.assess(group, news)
            sf = Superforecaster(ChatGPT)
            sf.forecast(event, news, rates, group)
            critic = Critic(HuggingChat)
            for ifp in group:
                print("Refining", ifp.question_id)
                for i in range(self.max_tries):
                    print("Pass", i, "of", self.max_tries, "on", ifp.question_id)
                    if 'concur' in ifp.feedback:
                        print("concur 1")
                        break
                    critic.feedback(ifp)
                    if 'concur' in ifp.feedback:
                        print("concur 2")
                        break
                    sf.reassess(ifp)
                print("===============================================")

    def report(self):
        for ifp in self.ifps.values():
            print(ifp.question_id, ifp.title)
            print("Forecast", ifp.forecast)
            print("Rationale", ifp.rationale, '\n')

    def upload(self):
        for ifp in self.ifps.values():
            ifp.upload()

In [41]:
if unit_test:
    test_day = '01AUG24'
    ifps = {id: IFP(id) for id in history[test_day]}

## Daily forecast

In [42]:
fcst = Forecaster(ifps)

In [43]:
fcst.fit()

MODEL DOMAIN
26957 Will Iran carry out a deadly attack within Israel before September 1, 2024?
**Model Domain: Military conflict**
27019 Will there be a debate between Kamala Harris and Donald Trump on September 4, 2024?
**Model Domain: Leadership change**
27020 Before October 1, 2024, will Anthropic announce on the news section of its website that it is planning an IPO?
**Model Domain: Dated product announcement**
27021 Will the USA win more Gold than Silver medals at the Paris 2024 Olympics?
**Model Domain: Sports performance**
27022 Will astronauts Suni Williams and Butch Wilmore be on Earth on September 15, 2024?
**Model Domain: Other**

CORRELATOR
|Iran-Israel military conflict|26957|Will Iran carry out a deadly attack within Israel before September 1, 2024?|
|US Presidential election|27019|Will there be a debate between Kamala Harris and Donald Trump on September 4, 2024?|
|Anthropic IPO|27020|Before October 1, 2024, will Anthropic announce on the news section of its website that

In [44]:
fcst.report()

26957 Will Iran carry out a deadly attack within Israel before September 1, 2024?
Forecast 75
Rationale I acknowledge the feedback regarding the complexity and potential for diplomatic interventions or changes in internal Iranian decision-making affecting the likelihood of an attack. Adjusting the probability to account for these variables provides a more nuanced assessment. While recent intelligence and military movements suggest a high likelihood of a retaliatory strike, it is prudent to temper confidence considering the dynamic nature of geopolitical events. A probability of 75% reflects a high likelihood while accommodating the inherent uncertainties. 

27019 Will there be a debate between Kamala Harris and Donald Trump on September 4, 2024?
Forecast 50
Rationale I appreciate the feedback. The lack of a confirmed agreement from both parties and the strong stance of the Harris campaign indeed weigh heavily on the probability. Considering these factors alongside the historical contex

In [45]:
fcst.upload()

Prediction posted for  26957
Comment posted for  26957
Prediction posted for  27019
Comment posted for  27019
Prediction posted for  27020
Comment posted for  27020
Prediction posted for  27021
Comment posted for  27021
Prediction posted for  27022
Comment posted for  27022
